In [ ]:
# 在图像分类领域内，其中的大杀器莫过于Resnet50了，这个残差神经网络当时被发明出来之后，顿时毁天灭地，
# 其余任何模型都无法想与之比拟。我们下面用Tensorflow来调用这个模型，
# 让我们的神经网络对Fashion-mnist数据集进行图像分类.由于在这个数据集当中图像的尺寸是28*28*1的，
# 如果想要使用resnet那就需要把28*28*1的灰度图变为224*224*3的RGB图，我们使用OpenCV库可以很容易将图像进行resize

In [ ]:
import os,sys
import numpy as np
import scipy
from scipy import ndimage
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from PIL import Image
import cv2 as cv
import random

In [ ]:
(train_image, train_label), (test_image, test_label) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# 读取数据集当中的500张图片
train_data = []
for img in train_image[:500]:
    resized_img = cv.resize(img, (224, 224))
    resized_img = cv.cvtColor(resized_img, cv.COLOR_GRAY2BGR)
    train_data.append(resized_img)

In [ ]:
# 我们最后得到了一个三维的列表数据，但是这并不是一个ndarray，也就是numpy当中的数组对象，
# 还无法进行训练，我们需要将其转化为numpy当中的数组，代码如下
train_data = np.array(train_data)
train_data.shape

In [ ]:
# 将数据进行归一化，加速神经网络的运算
train_data = train_data / 255.0

In [ ]:
# 编译和训练模型
model = ResNet50(weights=None, classes=10)
model.compile(optimizer="Adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(train_data, train_label[0:500], epochs=10, batch_size=6)
predictions = model.predict(test_image)

In [ ]:
plt.plot(history.epoch,history.history.get('loss'),label="loss")
plt.plot(history.epoch,history.history.get('val_loss'),label="val_loss")
plt.legend()

In [ ]:
def plot_image(i, predictions_array, true_label, img):
    predictions_array, true_label, img = predictions_array, true_label[i], img[i]
    plt.grid(False)
    plt.xticks([])
    plt.yticks([])

    plt.imshow(img, cmap=plt.cm.binary)

    predicted_label = np.argmax(predictions_array)
    if predicted_label == true_label:
        color = 'green'
    else:
        color = 'red'

    plt.xlabel("{} {:2.0f}% ({})".format(
        class_names[predicted_label], 
        100 * np.max(predictions_array),
        class_names[true_label]),
        color=color
    )


def plot_value_array(i, predictions_array, true_label):
    predictions_array, true_label = predictions_array, true_label[i]
    plt.grid(False)
    plt.xticks(range(10))
    plt.yticks([])
    thisplot = plt.bar(range(10), predictions_array, color="#777777")
    plt.ylim([0, 1])
    predicted_label = np.argmax(predictions_array)

    thisplot[predicted_label].set_color('red')
    thisplot[true_label].set_color('green')

In [ ]:
i = 1000
plt.figure(figsize=(6, 3))
plt.subplot(1, 2, 1)
plot_image(i, predictions[i], test_label, test_image)

plt.subplot(1, 2, 2)
plot_value_array(i, predictions[i], test_label)
plt.show()